<a href="https://colab.research.google.com/github/markpro-code/ml-homework/blob/main/chapter_02_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第二章作业

学生姓名: 毛军

以下是代码实现:

In [16]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [17]:
%%cython

import numpy as np
cimport numpy as np
import pandas as pd

# 数据初始化
dataSize = 5000000
y = np.random.randint(2, size=(dataSize, 1))
x = np.random.randint(10, size=(dataSize, 1))
testData = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

# 课上的例子
# =================================================================
cpdef target_mean_example(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
    cdef np.ndarray[double] x = np.asfortranarray(data[x_name], dtype=np.float64)

    target_mean_example_impl(result, y, x, nrow)
    return result

cdef void target_mean_example_impl(double[:] result, double[:] y, double[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()

    cdef long i
    for i in range(nrow):
        if x[i] not in value_dict.keys():
            value_dict[x[i]] = y[i]
            count_dict[x[i]] = 1
        else:
            value_dict[x[i]] += y[i]
            count_dict[x[i]] += 1

    i=0
    for i in range(nrow):
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)

# 我的代码实现
# =================================================================
cpdef target_mean_homework(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[int] y = np.asfortranarray(data[y_name], dtype=np.int32)
    cdef np.ndarray[int] x = np.asfortranarray(data[x_name], dtype=np.int32)

    target_mean_homework_impl(result, y, x, nrow)
    return result

cdef void target_mean_homework_impl(double[:] result, int[:] y, int[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()
    cdef long i = 0
    cdef int key

    while i < nrow:
        key = x[i]
        if key not in value_dict.keys():
            value_dict[key] = y[i]
            count_dict[key] = 1
        else:
            value_dict[key] += y[i]
            count_dict[key] += 1
        i += 1

    i=0
    while i < nrow:
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)
        i += 1

In [18]:
import time

print("测试数据量: " + str(dataSize))

start_time = time.time()
result = target_mean_example(testData, 'y', 'x')
print("--- 课上例子执行时间: %s seconds ---" % (time.time() - start_time))
print(result)

start_time = time.time()
result = target_mean_homework(testData, 'y', 'x')
print("--- 我的代码实现执行时间: %s seconds ---" % (time.time() - start_time))
print(result)

测试数据量: 5000000
--- 课上例子执行时间: 2.577223539352417 seconds ---
[0.50014923 0.50028786 0.50028786 ... 0.50067612 0.50084281 0.49967488]
--- 我的代码实现执行时间: 1.3680131435394287 seconds ---
[0.50014923 0.50028786 0.50028786 ... 0.50067612 0.50084281 0.49967488]


# 总结

此次主要优化点:
1. 将之前的 for loop 改成了 while loop, 省去了在 range 中遍历的过程 (个人猜测 python 在遍历 range 的时候有性能损失).
1. 将不必要 double 类型定义换成 int (也许分配更少的内存空间会快点)

优化结果: 500万条测试数据, 执行时间约 1.3s, 老师的例子执行时间约 2.5s, ***效率提升了约 1 倍***.
